In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import numpy as np
import random
import pickle 

def scrape_goodreads_ratings(user_id, max_pages=10):
    """
    Scrape a user's star ratings from Goodreads.
    
    Args:
    - user_id (str): Goodreads user ID or profile suffix.
    - max_pages (int): Maximum number of pages to scrape (each page contains ~30 books).
    
    Returns:
    - pd.DataFrame: A DataFrame containing book titles and ratings.
    """
    base_url = f"https://www.goodreads.com/review/list/{user_id}?shelf=read"
    headers = {"User-Agent": "Mozilla/5.0"}
    books = []

    for page in range(1, max_pages + 1):
        url = f"{base_url}&page={page}"
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.content, "html.parser")
        
        # Find all book entries in the table
        rows = soup.find_all("tr", class_="bookalike review")
        if not rows:
            print("No more data found.")
            break

        for row in rows:
            try:
                title = row.find("td", class_="field title").a.text.strip()
                rating_element = row.find("td", class_="field rating")
                rating = rating_element.find("span", class_="staticStars").get("title", "No rating")
                stars = map_rating(rating)
                books.append({"Title": title, "Rating": stars, "User_id": user_id})
#                 print(title, rating, stars)
            except AttributeError:
                # Handle rows with missing data
                continue

        print(f"Page {page} scraped successfully.")
        time.sleep(random.uniform(1, 5))  # Be kind to the server and avoid being blocked

    # Return data as a pandas DataFrame
    return pd.DataFrame(books)



In [2]:
def map_rating(phrase):
    rating_map = {
        "liked it": 3,
        "really liked it": 4,
        "it was ok": 2, 
        "it was amazing": 5, 
        "did not like it": 1,
    }
    
    return rating_map.get(phrase, "Invalid rating")  # Default to "Invalid rating" if the phrase isn't in the dictionary


In [3]:
# if __name__ == "__main__":
# #     user_id = "6688207"  # Replace with the Goodreads user ID or profile suffix
# #     for user_id in tqdm(['30181442', '75009563', '11345366', '110912303', '113964939', '11215896', '53701594', '4622890', '93628736', '176180116']):
# #     for user_id in tqdm(['2974095', '4622890', '28953843', '16174645', '4159922', '4125660', '54886546', '16912659', '260116', '4685500', '21865425']):
# #     for user_id in tqdm(['53701594', '27709782', '7566229', '16652861', '30817744', '56259255', '4125660', '60964126', 
# #                          '176167767', '28510930', '1029975', '131020767', '28862120', '88713906', '160141433', '41097916', 
# #                          '20809863', '69519261', '24017481', '7376365', '75941333', '13571407', '106618742', '17792052',
# #                          '3534528', '130656897', '7474475', '4125412', '6336365', '6026811', '3438047']):
#     for user_id in ['169695556']:
#         print("User_id = ", user_id)
#         max_pages = 30  # Adjust based on expected data
#         ratings_data = scrape_goodreads_ratings(user_id, max_pages)

#         if not ratings_data.empty:
# #             print(ratings_data.head())
# #             ratings_data.to_csv("goodreads_ratings.csv", index=False)
#             ratings_data.to_csv('goodreads_ratings.csv', mode='a', header=False, index=False)
#             print("Data saved to goodreads_ratings.csv.")
#         else:
#             print("No data retrieved.")


In [4]:
df = pd.read_csv('goodreads_ratings_series.csv')
print(df.shape)
df = df.drop_duplicates()
# Print the entire DataFrame
print(df)

duplicate_count = df['Title'].duplicated().sum()
print("Number of books with at least two people rating it:", duplicate_count)
duplicate_counts_per_value = df['Title'].value_counts()
print(duplicate_counts_per_value)
print("Number of unique books: ", df['Title'].nunique())
num_users = df['User_id'].nunique()
user_ids = list(df['User_id'].unique())
print("number of users is: ", num_users)
print("user_ids = ", user_ids)

(162199, 6)
                                                    Title Rating    User_id  \
0                                       I Am Watching You      3  169695558   
1       Three to Get Deadly\r\n        (Stephanie Plum...      3  169695558   
2       Before the Coffee Gets Cold\r\n        (Before...      4  169695558   
3       Dark Sacred Night\r\n        (Renée Ballard, #...      4  169695558   
4       Two for the Dough\r\n        (Stephanie Plum, #2)      4  169695558   
...                                                   ...    ...        ...   
162194                                  Souls of the Fire      5    4121938   
162195           Forget The Cures, Find The Cause, Book I      5    4121938   
162196                                               ILYR      5    4121938   
162197                              Butch: The Full Story      5    4121938   
162198                                           Butch II      5    4121938   

        Series  First  Suggest  
0     

In [5]:
# # Get a list of top titles in order
# top_titles = duplicate_counts_per_value.index.tolist()
# top_100 = top_titles[:100]

# for title in top_100:
#     print(title)
    
# with open("top_100.pkl", "wb") as file:
#     pickle.dump(top_100, file)

In [6]:
# threshold = 5#num_users * 0.1
# pop_titles = list(duplicate_counts_per_value[duplicate_counts_per_value > threshold].index)
# my_titles = df.loc[df["User_id"] == 169695558, "Title"].tolist()
# # print(my_titles)

# print("pop titles len = ", len(pop_titles))
# print(pop_titles)
# print("my titles len = ", len(my_titles))
# titles = list(set(pop_titles))# + my_titles))

# # #remove Harry Potter titles:
# # titles = [s for s in titles if "Harry Potter" not in s]

# num_titles = len(titles)


# print(titles)
# # print(titles)
# print("num_titles =", num_titles)

# # ratings = np.full((num_users, num_titles), None)
# ratings = np.zeros((num_users, num_titles))

# for index, row in df.iterrows():
#     if row['Title'] in titles:
#         try:
#             ratings[user_ids.index(row['User_id']), titles.index(row["Title"])] = int(row["Rating"])
# #             print("found ", row["Title"])
#         except:
#             pass
        
# print("ratings size = ", ratings.shape)
# ratings = ratings[~np.all(ratings == 0, axis=1)]
# print("ratings size = ", ratings.shape)
# # Save the list to a file
# with open("titles.pkl", "wb") as file:
#     pickle.dump(titles, file)

In [7]:
# # print(df['Suggest'])

# suggest = list(df['Suggest'])
# print(suggest)

# with open("suggest.pkl", "wb") as file:
#     suggest.dump(suggest, file)

In [8]:
# ratings_df = pd.DataFrame(ratings)
# print(ratings_df.shape)
# #delete users that don't have any of these ratings
# # ratings_df = ratings_df.loc[~(ratings_df == 0).all(axis=1)]
# ratings_df = ratings_df.loc[(ratings_df != 0).sum(axis=1) >= 4] #need at least 4 entries to stay
# print(ratings_df.shape)

# # Calculate percentage of non-zero elements
# percentage_nonzero = (np.count_nonzero(ratings_df) / ratings_df.size) * 100
# print("percentage_nonzero =", round(percentage_nonzero, 1), '%')

# # Save the list to a file
# with open("ratings_df.pkl", "wb") as file:
#     pickle.dump(ratings_df, file)


In [9]:
with open("titles.pkl", "rb") as file:
    titles = pickle.load(file)
    
with open("top_100.pkl", "rb") as file:
    top_100 = pickle.load(file)
    
with open("suggest.pkl", "rb") as file:
    suggest = pickle.load(file)
    
num_titles = len(titles)

# Load the list from the file
with open("ratings_df.pkl", "rb") as file:
    ratings_df = pickle.load(file)
    ratings = ratings_df.to_numpy()


In [10]:
# Convert the filled ratings data into a numpy array
ratings_matrix = ratings_df.values

# Initialize KNN (using user-based KNN)
import math
knn = NearestNeighbors(n_neighbors=50, metric='cosine')  # Using cosine similarity  math.ceil(num_users/10)
knn.fit(ratings_matrix)

with open("knn_model.pkl", "wb") as file:
    pickle.dump(knn, file)

# Example: Predict rating for User 0 on Item 2
user_id = 0  # Index of user in the matrix
item_id = 2  # Index of item to predict rating for

# Get the nearest neighbors for user 0 (excluding the user itself)
distances, indices = knn.kneighbors([ratings_matrix[user_id]])

pred_ratings_list = np.array([])
rankings_list = np.array([])
for item_id in range(num_titles):
    # Get the ratings for the neighbors on item 2
    neighbor_ratings = np.array([ratings_matrix[i, item_id] for i in indices[0] if not np.isnan(ratings_matrix[i, item_id])])

    predicted_rating = np.mean(neighbor_ratings[np.nonzero(neighbor_ratings)])
    rankings = np.sum(neighbor_ratings[np.nonzero(neighbor_ratings)])

#     print(predicted_rating)

    pred_ratings_list = np.append(pred_ratings_list, predicted_rating)
    rankings_list = np.append(rankings_list, rankings)
#     print(f"Predicted rating for User {user_id} on Item {item_id}: {predicted_rating}")
    
best_book_rating = np.max(pred_ratings_list)
best_book_idx = np.argmax(pred_ratings_list)

sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
n = 1
for i, idx in enumerate(sorted_indices): 
    if (ratings[0, idx] > 0) or (np.isnan(pred_ratings_list[idx])):
        pass
    else:
        print("#", (n) , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))
        n+=1

/root/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/root/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Top books are:
# 1 The Wicked King
        (The Folk of the Air, #2) Rating: 5.0
# 2 The Assassin's Blade
        (Throne of Glass, #0.1-0.5) Rating: 5.0
# 3 A Kingdom of Flesh and Fire
        (Blood and Ash, #2) Rating: 5.0
# 4 Knuffle Bunny: A Cautionary Tale
        (Knuffle Bunny, #1) Rating: 5.0
# 5 Little House in the Big Woods
        (Little House, #1) Rating: 5.0
# 6 Gathering Blue
        (The Giver, #2) Rating: 5.0
# 7 Cujo Rating: 5.0
# 8 Love You Forever Rating: 5.0
# 9 Blankets Rating: 5.0
# 10 Anne of the Island
        (Anne of Green Gables, #3) Rating: 5.0
# 11 The Amber Spyglass
        (His Dark Materials, #3) Rating: 5.0
# 12 Where the Crawdads Sing Rating: 5.0
# 13 The High King
        (The Chronicles of Prydain, #5) Rating: 5.0
# 14 Animal Farm / 1984 Rating: 5.0
# 15 Hannibal
        (Hannibal Lecter, #3) Rating: 5.0
# 16 The Subtle Knife
        (His Dark Materials, #2) Rating: 5.0
# 17 Ruthless Vows
        (Letters of Enchantment, #2) Rating: 5.0
# 18 The Bu

# 171 The Hero and the Crown
        (Damar, #2) Rating: 5.0
# 172 The Little House Collection
        (Little House, #1-9) Rating: 5.0
# 173 Funny Story Rating: 5.0
# 174 Physik
        (Septimus Heap, #3) Rating: 5.0
# 175 Beneath a Scarlet Sky Rating: 5.0
# 176 Saga, Volume 1 Rating: 5.0
# 177 Fablehaven
        (Fablehaven, #1) Rating: 5.0
# 178 The Lucky One Rating: 5.0
# 179 Harold and the Purple Crayon
        (Harold, #1) Rating: 5.0
# 180 The Forest of Hands and Teeth
        (The Forest of Hands and Teeth, #1) Rating: 5.0
# 181 The Dragon Reborn
        (The Wheel of Time, #3) Rating: 5.0
# 182 The Boxcar Children
        (The Boxcar Children, #1) Rating: 5.0
# 183 Wish You Well Rating: 5.0
# 184 Marley and Me: Life and Love With the Worldâ€™s Worst Dog Rating: 5.0
# 185 The Clique
        (The Clique, #1) Rating: 5.0
# 186 Sharp Objects Rating: 5.0
# 187 Jamaica Inn Rating: 5.0
# 188 Dreams of Gods & Monsters
        (Daughter of Smoke & Bone, #3) Rating: 5.0
# 189 House of 

# 355 The Namesake Rating: 4.0
# 356 A Good Girl's Guide to Murder
        (A Good Girl's Guide to Murder, #1) Rating: 4.0
# 357 The Titanâ€™s Curse
        (Percy Jackson and the Olympians, #3) Rating: 4.0
# 358 Forever Odd
        (Odd Thomas, #2) Rating: 4.0
# 359 Circe Rating: 4.0
# 360 Prince Caspian
        (Chronicles of Narnia, #2) Rating: 4.0
# 361 The Other Boleyn Girl
        (The Plantagenet and Tudor Novels, #9) Rating: 4.0
# 362 The Ugly Truth
        (Diary of a Wimpy Kid, #5) Rating: 4.0
# 363 Tempted
        (House of Night, #6) Rating: 4.0
# 364 The Silent Patient Rating: 4.0
# 365 Bridget Jones: The Edge of Reason
        (Bridget Jones, #2) Rating: 4.0
# 366 Odd Thomas
        (Odd Thomas, #1) Rating: 4.0
# 367 The Screwtape Letters Rating: 4.0
# 368 Little Dorrit Rating: 4.0
# 369 Demonglass
        (Hex Hall, #2) Rating: 4.0
# 370 The Night Before Christmas Rating: 4.0
# 371 4th of July
        (Women's Murder Club, #4) Rating: 4.0
# 372 The Three Musketeers Ratin

# 537 The Last Song Rating: 4.0
# 538 Marked
        (House of Night, #1) Rating: 4.0
# 539 Brown Bear, Brown Bear, What Do You See? Rating: 4.0
# 540 Rodrick Rules
        (Diary of a Wimpy Kid, #2) Rating: 4.0
# 541 Mansfield Park Rating: 4.0
# 542 The Last Straw
        (Diary of a Wimpy Kid, #3) Rating: 4.0
# 543 Redwall
        (Redwall, #1) Rating: 4.0
# 544 1st to Die
        (Women's Murder Club, #1) Rating: 4.0
# 545 Baby Proof Rating: 4.0
# 546 A ​Court of Silver Flames
        (A Court of Thorns and Roses, #4) Rating: 4.0
# 547 And Then There Were None Rating: 4.0
# 548 Go the Fuck to Sleep Rating: 4.0
# 549 Eats, Shoots & Leaves: The Zero Tolerance Approach to Punctuation Rating: 4.0
# 550 The Iron King
        (The Iron Fey, #1) Rating: 4.0
# 551 Calvin and Hobbes
        (Calvin and Hobbes, #1) Rating: 4.0
# 552 I'll Be Gone in the Dark: One Woman's Obsessive Search for the Golden State Killer Rating: 4.0
# 553 In Cold Blood Rating: 4.0
# 554 Midnight Sun
        (The Twi

# 825 Lucky Rating: 2.0
# 826 The Shack Rating: 1.0
# 827 Snuff Rating: 1.0
# 828 Twilight: The Complete Illustrated Movie Companion Rating: 1.0
# 829 Lies My Teacher Told Me: Everything Your American History Textbook Got Wrong Rating: 1.0
# 830 Catch-22 Rating: 1.0
# 831 I Know Why the Caged Bird Sings
        (Maya Angelou's Autobiography, #1) Rating: 1.0
# 832 Running with Scissors Rating: 1.0
# 833 The Other Queen
        (The Plantagenet and Tudor Novels, #15) Rating: 1.0
# 834 Cutting for Stone Rating: 1.0
# 835 The Audacity of Hope: Thoughts on Reclaiming the American Dream Rating: 1.0
# 836 Moby-Dick or, The Whale Rating: 1.0
# 837 Moby Dick Rating: 1.0
# 838 A Portrait of the Artist as a Young Man Rating: 1.0
# 839 We the Animals Rating: 1.0


In [11]:
#make knn for the similar user part
# Initialize KNN (using user-based KNN)
# import math
knn_30 = NearestNeighbors(n_neighbors=50, metric='cosine')  # Using cosine similarity
knn_30.fit(ratings_matrix)

with open("knn_model_30.pkl", "wb") as file:
    pickle.dump(knn_30, file)

In [12]:
rankings_list

best_book_rating = np.max(rankings_list)
best_book_idx = np.argmax(rankings_list)

sorted_indices = np.argsort(rankings_list)[::-1]
print("Top books are:")
for i, idx in enumerate(sorted_indices): 
    print("#", (i+1) , titles[idx], "Rating:", round(pred_ratings_list[idx], 1), ". Ranking:", rankings_list[idx])

Top books are:
# 1 Harry Potter and the Goblet of Fire
        (Harry Potter, #4) Rating: 4.8 . Ranking: 216.0
# 2 Harry Potter and the Deathly Hallows
        (Harry Potter, #7) Rating: 4.8 . Ranking: 216.0
# 3 Harry Potter and the Prisoner of Azkaban
        (Harry Potter, #3) Rating: 4.7 . Ranking: 216.0
# 4 Harry Potter and the Half-Blood Prince
        (Harry Potter, #6) Rating: 4.6 . Ranking: 209.0
# 5 Harry Potter and the Order of the Phoenix
        (Harry Potter, #5) Rating: 4.5 . Ranking: 198.0
# 6 Harry Potter and the Sorcerer's Stone
        (Harry Potter, #1) Rating: 4.7 . Ranking: 196.0
# 7 Harry Potter and the Chamber of Secrets
        (Harry Potter, #2) Rating: 4.3 . Ranking: 195.0
# 8 The Hunger Games
        (The Hunger Games, #1) Rating: 4.7 . Ranking: 149.0
# 9 Catching Fire
        (The Hunger Games, #2) Rating: 4.5 . Ranking: 117.0
# 10 Mockingjay
        (The Hunger Games, #3) Rating: 4.3 . Ranking: 113.0
# 11 Twilight
        (The Twilight Saga, #1) Rating: 4.1

# 157 Dark Places Rating: 3.7 . Ranking: 11.0
# 158 Confessions of a Shopaholic
        (Shopaholic, #1) Rating: 3.7 . Ranking: 11.0
# 159 Ready Player One
        (Ready Player One, #1) Rating: 3.7 . Ranking: 11.0
# 160 The Amulet of Samarkand
        (Bartimaeus, #1) Rating: 3.7 . Ranking: 11.0
# 161 Lean Mean Thirteen
        (Stephanie Plum, #13) Rating: 3.7 . Ranking: 11.0
# 162 Hard Eight
        (Stephanie Plum, #8) Rating: 3.7 . Ranking: 11.0
# 163 The Cat in the Hat
        (Cat in the Hat, #1) Rating: 3.7 . Ranking: 11.0
# 164 Wicked: The Life and Times of the Wicked Witch of the West
        (The Wicked Years, #1) Rating: 3.7 . Ranking: 11.0
# 165 Tuesdays with Morrie Rating: 3.7 . Ranking: 11.0
# 166 Hot Six
        (Stephanie Plum, #6) Rating: 3.7 . Ranking: 11.0
# 167 Eleven on Top
        (Stephanie Plum, #11) Rating: 3.7 . Ranking: 11.0
# 168 Love in the Time of Cholera Rating: 3.7 . Ranking: 11.0
# 169 High Five
        (Stephanie Plum, #5) Rating: 3.7 . Ranking: 11.0


# 301 Horton Hears a Who! Rating: 4.0 . Ranking: 8.0
# 302 Fear Nothing
        (Moonlight Bay, #1) Rating: 4.0 . Ranking: 8.0
# 303 Their Eyes Were Watching God Rating: 4.0 . Ranking: 8.0
# 304 The Memory Keeper's Daughter Rating: 4.0 . Ranking: 8.0
# 305 The Screwtape Letters Rating: 4.0 . Ranking: 8.0
# 306 Fast Food Nation: The Dark Side of the All-American Meal Rating: 4.0 . Ranking: 8.0
# 307 Plum Lovin'
        (Stephanie Plum, #12.5) Rating: 2.7 . Ranking: 8.0
# 308 The Immortal Life of Henrietta Lacks Rating: 4.0 . Ranking: 8.0
# 309 Speak Rating: 4.0 . Ranking: 8.0
# 310 Matilda Rating: 4.0 . Ranking: 8.0
# 311 Dune
        (Dune, #1) Rating: 4.0 . Ranking: 8.0
# 312 Bridget Jones: The Edge of Reason
        (Bridget Jones, #2) Rating: 4.0 . Ranking: 8.0
# 313 Into the Wilderness
        (Wilderness, #1) Rating: 4.0 . Ranking: 8.0
# 314 The Iron Queen
        (The Iron Fey, #3) Rating: 4.0 . Ranking: 8.0
# 315 The Great Hunt
        (The Wheel of Time, #2) Rating: 4.0 . Ranki

# 451 Through the Woods Rating: 5.0 . Ranking: 5.0
# 452 Z: A Novel of Zelda Fitzgerald Rating: 5.0 . Ranking: 5.0
# 453 My Ántonia Rating: 5.0 . Ranking: 5.0
# 454 Rebecca Rating: 5.0 . Ranking: 5.0
# 455 Steelheart
        (The Reckoners, #1) Rating: 5.0 . Ranking: 5.0
# 456 The Name of the Wind
        (The Kingkiller Chronicle, #1) Rating: 5.0 . Ranking: 5.0
# 457 The Last Battle
        (Chronicles of Narnia, #7) Rating: 5.0 . Ranking: 5.0
# 458 The Best of Me Rating: 5.0 . Ranking: 5.0
# 459 Hyperbole and a Half: Unfortunate Situations, Flawed Coping Mechanisms, Mayhem, and Other Things That Happened Rating: 5.0 . Ranking: 5.0
# 460 Maus: A Survivor's Tale II: And Here My Troubles Began
        (Maus, #2) Rating: 5.0 . Ranking: 5.0
# 461 Saga, Volume 2 Rating: 5.0 . Ranking: 5.0
# 462 The Silence of the Lambs
        (Hannibal Lecter, #2) Rating: 5.0 . Ranking: 5.0
# 463 One Day in the Life of Ivan Denisovich Rating: 5.0 . Ranking: 5.0
# 464 The Mist Rating: 5.0 . Ranking: 5.0
# 

# 623 Americanah Rating: 4.0 . Ranking: 4.0
# 624 High Fidelity Rating: 4.0 . Ranking: 4.0
# 625 The Partner Rating: 4.0 . Ranking: 4.0
# 626 Orphan Train Rating: 4.0 . Ranking: 4.0
# 627 Fox in Socks Rating: 4.0 . Ranking: 4.0
# 628 Sing You Home Rating: 4.0 . Ranking: 4.0
# 629 The Tommyknockers Rating: 4.0 . Ranking: 4.0
# 630 The Choice Rating: 4.0 . Ranking: 4.0
# 631 Handle with Care Rating: 4.0 . Ranking: 4.0
# 632 The Three Musketeers Rating: 4.0 . Ranking: 4.0
# 633 Ross Poldark
        (Poldark, #1) Rating: 4.0 . Ranking: 4.0
# 634 The Fires of Heaven
        (The Wheel of Time, #5) Rating: 4.0 . Ranking: 4.0
# 635 Superfudge
        (Fudge, #3) Rating: 4.0 . Ranking: 4.0
# 636 The BFG Rating: 4.0 . Ranking: 4.0
# 637 The Innocent Rating: 4.0 . Ranking: 4.0
# 638 Doctor Sleep
        (The Shining, #2) Rating: 4.0 . Ranking: 4.0
# 639 The Tale of Peter Rabbit
        (World of Beatrix Potter, #1) Rating: 4.0 . Ranking: 4.0
# 640 No Second Chance Rating: 4.0 . Ranking: 4.0
# 64

...

# 3454 The Stranger Diaries
        (Harbinder Kaur, #1) Rating: nan . Ranking: 0.0
# 3455 â€™Salemâ€™s Lot Rating: nan . Ranking: 0.0
# 3456 The Space Between Us Rating: nan . Ranking: 0.0
# 3457 The Goal
        (Off-Campus, #4) Rating: nan . Ranking: 0.0
# 3458 Alone
        (Detective D.D. Warren, #1) Rating: nan . Ranking: 0.0
# 3459 The Ocean at the End of the Lane Rating: nan . Ranking: 0.0
# 3460 Such a Fun Age Rating: nan . Ranking: 0.0
# 3461 Shadow of Night
        (All Souls, #2) Rating: nan . Ranking: 0.0
# 3462 Endurance: Shackleton's Incredible Voyage Rating: nan . Ranking: 0.0
# 3463 P.S. I Still Love You
        (To All the Boys I've Loved Before, #2) Rating: nan . Ranking: 0.0
# 3464 Leviathan
        (Leviathan, #1) Rating: nan . Ranking: 0.0
# 3465 The Light We Carry: Overcoming in Uncertain Times Rating: nan . Ranking: 0.0
# 3466 Becoming Rating: nan . Ranking: 0.0
# 3467 It's Kind of a Funny Story Rating: nan . Ranking: 0.0
# 3468 Second Foundation
        (Founda

# 3598 The Good House Rating: nan . Ranking: 0.0
# 3599 Pippi Longstocking
        (Pippi Långstrump, #1) Rating: nan . Ranking: 0.0
# 3600 Everything is Illuminated Rating: nan . Ranking: 0.0
# 3601 The Art of Loving Rating: nan . Ranking: 0.0
# 3602 Before I Fall Rating: nan . Ranking: 0.0
# 3603 Norse Mythology Rating: nan . Ranking: 0.0
# 3604 The Distant Hours Rating: nan . Ranking: 0.0
# 3605 A Tale for the Time Being Rating: nan . Ranking: 0.0
# 3606 The Secret Adversary
        (Tommy and Tuppence Mysteries, #1) Rating: nan . Ranking: 0.0
# 3607 The Concrete Blonde
        (Harry Bosch, #3; Harry Bosch Universe, #3) Rating: nan . Ranking: 0.0
# 3608 Secret Daughter Rating: nan . Ranking: 0.0
# 3609 There There Rating: nan . Ranking: 0.0
# 3610 The Well of Ascension
        (Mistborn, #2) Rating: nan . Ranking: 0.0
# 3611 Sunshine Rating: nan . Ranking: 0.0
# 3612 A Storm of Swords 2: Blood and Gold
        (A Song of Ice and Fire, #3, Part 2 of 2) Rating: nan . Ranking: 0.0
# 3

        (The Ruinous Love Trilogy, #1) Rating: nan . Ranking: 0.0
# 3750 Airborn
        (Matt Cruse, #1) Rating: nan . Ranking: 0.0
# 3751 Fool Rating: nan . Ranking: 0.0
# 3752 The Shadows Rating: nan . Ranking: 0.0
# 3753 The Bookish Life of Nina Hill Rating: nan . Ranking: 0.0
# 3754 An Ember in the Ashes
        (An Ember in the Ashes, #1) Rating: nan . Ranking: 0.0
# 3755 The Winemaker's Wife Rating: nan . Ranking: 0.0
# 3756 The Splendid and the Vile: A Saga of Churchill, Family, and Defiance During the Blitz Rating: nan . Ranking: 0.0
# 3757 Angels Flight
        (Harry Bosch, #6; Harry Bosch Universe, #8) Rating: nan . Ranking: 0.0
# 3758 Before We Were Yours Rating: nan . Ranking: 0.0
# 3759 The Amazing Adventures of Kavalier & Clay Rating: nan . Ranking: 0.0
# 3760 Martin Eden Rating: nan . Ranking: 0.0
# 3761 How to Be Good Rating: nan . Ranking: 0.0
# 3762 A Fall of Marigolds Rating: nan . Ranking: 0.0
# 3763 The Gathering Storm
        (The Wheel of Time, #12) Rating: nan

# 3891 Colorless Tsukuru Tazaki and His Years of Pilgrimage Rating: nan . Ranking: 0.0
# 3892 Confessions on the 7:45 Rating: nan . Ranking: 0.0
# 3893 The Vacancy in Room 10 Rating: nan . Ranking: 0.0
# 3894 She Said: Breaking the Sexual Harassment Story That Helped Ignite a Movement Rating: nan . Ranking: 0.0
# 3895 Lincoln in the Bardo Rating: nan . Ranking: 0.0
# 3896 The Angel Maker Rating: nan . Ranking: 0.0
# 3897 The Last Flight Rating: nan . Ranking: 0.0
# 3898 Pyramids
        (Discworld, #7) Rating: nan . Ranking: 0.0
# 3899 Soul Harvest: The World Takes Sides
        (Left Behind, #4) Rating: nan . Ranking: 0.0
# 3900 Luckiest Girl Alive Rating: nan . Ranking: 0.0
# 3901 A Confederacy of Dunces Rating: nan . Ranking: 0.0
# 3902 The Blind Assassin Rating: nan . Ranking: 0.0
# 3903 True Colors Rating: nan . Ranking: 0.0
# 3904 Daring Greatly: How the Courage to Be Vulnerable Transforms the Way We Live, Love, Parent, and Lead Rating: nan . Ranking: 0.0
# 3905 The Tea Girl of H

# 4064 The Door Rating: nan . Ranking: 0.0
# 4065 Gone Tonight Rating: nan . Ranking: 0.0
# 4066 Secluded Cabin Sleeps Six Rating: nan . Ranking: 0.0
# 4067 Hour Game
        (Sean King & Michelle Maxwell, #2) Rating: nan . Ranking: 0.0
# 4068 Finlay Donovan Jumps the Gun
        (Finlay Donovan, #3) Rating: nan . Ranking: 0.0
# 4069 The Professor and the Madman: A Tale of Murder, Insanity and the Making of the Oxford English Dictionary Rating: nan . Ranking: 0.0
# 4070 The Comedy of Errors Rating: nan . Ranking: 0.0
# 4071 Same Time Next Summer Rating: nan . Ranking: 0.0
# 4072 A Midsummer Nightâ€™s Dream Rating: nan . Ranking: 0.0
# 4073 I Am Pilgrim
        (Pilgrim, #1) Rating: nan . Ranking: 0.0
# 4074 The Lager Queen of Minnesota Rating: nan . Ranking: 0.0
# 4075 The Jungle Book
        (Jungle Book, #1) Rating: nan . Ranking: 0.0
# 4076 Girl, Woman, Other Rating: nan . Ranking: 0.0
# 4077 Caught Up
        (Windy City, #3) Rating: nan . Ranking: 0.0
# 4078 Sam Rating: nan . Rank

In [13]:
#give a list sorted out with books you've already read:
# sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
list_num = 1
for idx in sorted_indices: 
#     print("ratings_matrix[user_id, idx]= ", ratings_matrix[user_id, idx])
    if (ratings_matrix[user_id, idx] > 0) or (np.isnan(pred_ratings_list[idx])):
        continue
    print("#", list_num , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))
    list_num += 1

Top books are:
# 1 Harry Potter and the Sorcerer's Stone
        (Harry Potter, #1) Rating: 4.7
# 2 Eclipse
        (The Twilight Saga, #3) Rating: 4.3
# 3 The da Vinci Code
        (Robert Langdon, #2) Rating: 4.5
# 4 The Great Gatsby Rating: 3.7
# 5 Pride and Prejudice Rating: 4.6
# 6 Breaking Dawn
        (The Twilight Saga, #4) Rating: 4.2
# 7 The Hobbit, or There and Back Again
        (The Lord of the Rings, #0) Rating: 4.6
# 8 The Kite Runner Rating: 4.6
# 9 The Fellowship of the Ring
        (The Lord of the Rings, #1) Rating: 4.0
# 10 The Help Rating: 4.8
# 11 The Sea of Monsters
        (Percy Jackson and the Olympians, #2) Rating: 4.1
# 12 The Diary of a Young Girl Rating: 4.9
# 13 The Lion, the Witch and the Wardrobe
        (Chronicles of Narnia, #1) Rating: 4.2
# 14 The Lovely Bones Rating: 3.3
# 15 The Last Olympian
        (Percy Jackson and the Olympians, #5) Rating: 4.7
# 16 The Host
        (The Host, #1) Rating: 4.7
# 17 The Battle of the Labyrinth
        (Percy Ja

        (Sword of Truth, #9) Rating: 5.0
# 157 Stone of Tears
        (Sword of Truth, #2) Rating: 5.0
# 158 Sizzling Sixteen
        (Stephanie Plum, #16) Rating: 3.3
# 159 Fablehaven
        (Fablehaven, #1) Rating: 5.0
# 160 Vampire Academy
        (Vampire Academy, #1) Rating: 5.0
# 161 Fearless Fourteen
        (Stephanie Plum, #14) Rating: 3.3
# 162 Hannibal
        (Hannibal Lecter, #3) Rating: 5.0
# 163 Miss Peregrine's Home for Peculiar Children
        (Miss Peregrine's Peculiar Children, #1) Rating: 5.0
# 164 Ten Big Ones
        (Stephanie Plum, #10) Rating: 3.3
# 165 Into the Wild Rating: 3.3
# 166 Room Rating: 5.0
# 167 The Girl Who Played with Fire
        (Millennium, #2) Rating: 5.0
# 168 The Invention of Hugo Cabret Rating: 5.0
# 169 Phantom
        (Sword of Truth, #10) Rating: 5.0
# 170 The Wicked King
        (The Folk of the Air, #2) Rating: 5.0
# 171 The Burning Bridge
        (Ranger's Apprentice, #2) Rating: 5.0
# 172 Unearthly
        (Unearthly, #1) Rating: 5

# 347 Hannibal Rising
        (Hannibal Lecter, #4) Rating: 5.0
# 348 A Bend in the Road Rating: 5.0
# 349 The Constant Princess
        (The Plantagenet and Tudor Novels, #6) Rating: 5.0
# 350 Dreams of Gods & Monsters
        (Daughter of Smoke & Bone, #3) Rating: 5.0
# 351 Fire
        (Graceling Realm, #2) Rating: 5.0
# 352 The Kill Artist
        (Gabriel Allon, #1) Rating: 5.0
# 353 The Thorn Birds Rating: 5.0
# 354 Blankets Rating: 5.0
# 355 It Starts with Us
        (It Ends with Us, #2) Rating: 5.0
# 356 Xenocide
        (Ender's Saga, #3) Rating: 5.0
# 357 Nine Stories Rating: 5.0
# 358 The Marriage Bargain
        (Marriage to a Billionaire, #1) Rating: 5.0
# 359 Jamaica Inn Rating: 5.0
# 360 November 9 Rating: 5.0
# 361 Anne of Green Gables
        (Anne of Green Gables, #1) Rating: 5.0
# 362 Dragon Rider
        (Dragon Rider, #1) Rating: 5.0
# 363 Anne's House of Dreams
        (Anne of Green Gables, #5) Rating: 5.0
# 364 Children of the Mind
        (Ender's Saga, #4) Ra

        (The Dark Tower, #4) Rating: 4.0
# 531 Savannah Blues
        (Weezie and Bebe Mysteries, #1) Rating: 4.0
# 532 Holidays on Ice Rating: 4.0
# 533 Ugly Love Rating: 4.0
# 534 The Shadow Rising
        (The Wheel of Time, #4) Rating: 4.0
# 535 Untamed Rating: 4.0
# 536 The Waste Lands
        (The Dark Tower, #3) Rating: 4.0
# 537 Tess of the D’Urbervilles Rating: 4.0
# 538 11/22/63 Rating: 4.0
# 539 5th Horseman
        (Women's Murder Club, #5) Rating: 4.0
# 540 Silence
        (Hush, Hush, #3) Rating: 4.0
# 541 The Hound of the Baskervilles
        (Sherlock Holmes, #5) Rating: 4.0
# 542 Moonwalking with Einstein: The Art and Science of Remembering Everything Rating: 4.0
# 543 Invisible Monsters Rating: 4.0
# 544 Many Waters
        (Time Quintet, #4) Rating: 4.0
# 545 The Spirit Catches You and You Fall Down: A Hmong Child, Her American Doctors, and the Collision of Two Cultures Rating: 4.0
# 546 Before We Were Innocent Rating: 4.0
# 547 Tiny Beautiful Things: Advice on Love 

# 709 House Rules Rating: 3.0
# 710 Storm Front
        (The Dresden Files, #1) Rating: 3.0
# 711 The Quickie Rating: 3.0
# 712 The Selection
        (The Selection, #1) Rating: 3.0
# 713 Hamnet Rating: 3.0
# 714 The Mayor of Casterbridge Rating: 3.0
# 715 Murder on the Orient Express
        (Hercule Poirot, #10) Rating: 3.0
# 716 Educated Rating: 3.0
# 717 Flat-Out Love
        (Flat-Out Love, #1) Rating: 3.0
# 718 Needful Things Rating: 3.0
# 719 The City of Ember
        (Book of Ember, #1) Rating: 3.0
# 720 Tess of the Dâ€™Urbervilles Rating: 3.0
# 721 And the Mountains Echoed Rating: 3.0
# 722 The Death of Ivan Ilych Rating: 3.0
# 723 The Remains of the Day Rating: 3.0
# 724 The Story of Babar
        (Babar, #1) Rating: 3.0
# 725 Little Fires Everywhere Rating: 3.0
# 726 Dad Is Fat Rating: 3.0
# 727 The Prince Rating: 3.0
# 728 Walk Two Moons Rating: 3.0
# 729 O Pioneers! Rating: 3.0
# 730 The Diving Bell and the Butterfly: A Memoir of Life in Death Rating: 3.0
# 731 Bird Box
  

In [14]:
idx = 8
this_ratings = ratings_matrix[indices[0,idx]]
print(this_ratings)

my_ratings = ratings_matrix[indices[0,0]]
print(my_ratings)

for i, rating in enumerate(this_ratings):
    if rating > 0 and my_ratings[i]>0:
        print("-", titles[i], ", their Rating:", rating, " My Rating:", my_ratings[i])

# print(distances, indices[-1,-1])

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
- Princess Academy
        (Princess Academy, #1) , their Rating: 5.0  My Rating: 4.0
- Twilight
        (The Twilight Saga, #1) , their Rating: 5.0  My Rating: 3.0
- The Giver
        (The Giver, #1) , their Rating: 3.0  My Rating: 4.0
- Harry Potter and the Deathly Hallows
        (Harry Potter, #7) , their Rating: 5.0  My Rating: 5.0
- Mockingjay
        (The Hunger Games, #3) , their Rating: 4.0  My Rating: 5.0
- Harry Potter and the Order of the Phoenix
        (Harry Potter, #5) , their Rating: 5.0  My Rating: 5.0
- Catching Fire
        (The Hunger Games, #2) , their Rating: 5.0  My Rating: 5.0
- Divergent
        (Divergent, #1) , their Rating: 5.0  My Rating: 5.0
- New Moon
        (The Twilight Saga, #2) , their Rating: 5.0  My Rating: 3.0
- Harry Potter and the Chamber of Secrets
        (Harry Potter, #2) , their Rating: 5.0  My Rating: 5.0
- Eragon
        (The Inheritance Cycle, #1) , their Rating: 5.0  My Rating: 5.0
- Harr

In [15]:
#find most similar books using cosine similarity
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(ratings_matrix.T)

similarity_df = pd.DataFrame(similarity_matrix, index=titles, columns=titles)

# Function to get k nearest neighbors for a movie
def get_similar_book(book_name, k=3):
    similar_book = similarity_df[book_name].sort_values(ascending=False)[1:k+1]
    return similar_book

book_name = 'First Lie Wins'
print("\nTop 5 similar book to", book_name, ":")
print(get_similar_book(book_name, k=5))


Top 5 similar book to First Lie Wins :
Listen for the Lie             0.647835
Darling Girls                  0.629509
She's Not Sorry                0.618190
The Fury                       0.576752
The Last One at the Wedding    0.574207
Name: First Lie Wins, dtype: float64


In [16]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

# Example user rating data (rows = users, columns = items)
ratings_df = pd.DataFrame(ratings)

# Step 1: Handle missing values using imputation (we will fill missing values with the mean rating)
imputer = SimpleImputer(strategy='mean')
ratings_filled = imputer.fit_transform(ratings_df)

# Step 2: Apply KMeans clustering
# We will use 2 clusters as an example
kmeans = KMeans(n_clusters=int(num_users/10), random_state=42)
clusters = kmeans.fit_predict(ratings_filled)

# Step 3: Add the cluster labels to the original DataFrame
ratings_df['Cluster'] = clusters

# # Print the user ratings with cluster assignments
# print("\nUser Ratings with Clusters:")
# print(ratings_df)

# # Step 4: Print the cluster centers (the centroid of each cluster)
# print("\nCluster Centers (Centroids):")
# print(kmeans.cluster_centers_)


user_id = 0
# print(clusters)

cluster_this_user = clusters[user_id]
# print(cluster_this_user)

pred_ratings_list = kmeans.cluster_centers_[user_id]
# print(pred_ratings_list)
# for i in range(len())

#give a list sorted out with books you've already read:
sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
list_num = 1
for idx in sorted_indices: 
#     print("ratings_matrix[user_id, idx]= ", ratings_matrix[user_id, idx])
    if (ratings_matrix[user_id, idx] > 0) or (np.isnan(pred_ratings_list[idx])) :
        continue
    print("#", list_num , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))
    list_num += 1

/root/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Top books are:
# 1 Blood Rites
        (The Dresden Files, #6) Rating: 5.0
# 2 The Camel Club
        (The Camel Club, #1) Rating: 5.0
# 3 Field of Dishonor
        (Honor Harrington, #4) Rating: 5.0
# 4 Without Fail
        (Jack Reacher, #6) Rating: 5.0
# 5 Fool Moon
        (The Dresden Files, #2) Rating: 5.0
# 6 Captain's Fury
        (Codex Alera, #4) Rating: 5.0
# 7 The Honor of the Queen
        (Honor Harrington, #2) Rating: 5.0
# 8 Absolute Power Rating: 5.0
# 9 Summer Knight
        (The Dresden Files, #4) Rating: 5.0
# 10 Die Trying
        (Jack Reacher, #2) Rating: 5.0
# 11 Hour Game
        (Sean King & Michelle Maxwell, #2) Rating: 5.0
# 12 The Hard Way
        (Jack Reacher, #10) Rating: 5.0
# 13 Simple Genius
        (Sean King & Michelle Maxwell, #3) Rating: 5.0
# 14 Echo Burning
        (Jack Reacher, #5) Rating: 5.0
# 15 Nothing to Lose
        (Jack Reacher, #12) Rating: 5.0
# 16 Cursor's Fury
        (Codex Alera, #3) Rating: 5.0
# 17 First Lord's Fury
        (Co

# 196 Fearless Fourteen
        (Stephanie Plum, #14) Rating: 0.0
# 197 The Three Billy Goats Gruff Rating: 0.0
# 198 Running with Scissors Rating: 0.0
# 199 The Tale of Despereaux Rating: 0.0
# 200 Before the Fall Rating: 0.0
# 201 Unearthly
        (Unearthly, #1) Rating: 0.0
# 202 Manâ€™s Search for Meaning Rating: 0.0
# 203 West With Giraffes Rating: 0.0
# 204 The Brutal Telling
        (Chief Inspector Armand Gamache #5) Rating: 0.0
# 205 Catch and Kill: Lies, Spies, and a Conspiracy to Protect Predators Rating: 0.0
# 206 Into Thin Air: A Personal Account of the Mt. Everest Disaster Rating: 0.0
# 207 Betrayed
        (House of Night, #2) Rating: 0.0
# 208 The Girl Who Chased the Moon Rating: 0.0
# 209 Under the Whispering Door Rating: 0.0
# 210 The Elite
        (The Selection, #2) Rating: 0.0
# 211 Holidays on Ice Rating: 0.0
# 212 The Mistake
        (Off-Campus, #2) Rating: 0.0
# 213 The Rules of Magic
        (Practical Magic, #0.2) Rating: 0.0
# 214 The Book Thief Rating: 0.0

        (Tarzan, #1) Rating: 0.0
# 360 Who Moved My Cheese? An Amazing Way to Deal with Change in Your Work and in Your Life... Rating: 0.0
# 361 The Selfish Gene Rating: 0.0
# 362 The Once and Future King Rating: 0.0
# 363 White Cat
        (Curse Workers, #1) Rating: 0.0
# 364 Spinning Silver Rating: 0.0
# 365 Eclipse
        (The Twilight Saga, #3) Rating: 0.0
# 366 Look Alive Twenty-Five
        (Stephanie Plum, #25) Rating: 0.0
# 367 No Second Chance Rating: 0.0
# 368 The Pram Rating: 0.0
# 369 The Innocent
        (Will Robie, #1) Rating: 0.0
# 370 The Arctic Incident
        (Artemis Fowl, #2) Rating: 0.0
# 371 The Enchanted Rating: 0.0
# 372 Our Souls at Night Rating: 0.0
# 373 Saga, Volume 1 Rating: 0.0
# 374 The Nix Rating: 0.0
# 375 Charlotteâ€™s Web Rating: 0.0
# 376 The Twist of a Knife
        (Hawthorne & Horowitz #4) Rating: 0.0
# 377 The Great Hunt
        (The Wheel of Time, #2) Rating: 0.0
# 378 Carmilla Rating: 0.0
# 379 The Family Upstairs
        (The Family Upsta

# 561 The Best of Me Rating: 0.0
# 562 The Lies of Locke Lamora
        (Gentleman Bastard, #1) Rating: 0.0
# 563 Cyrano de Bergerac Rating: 0.0
# 564 Haunted Rating: 0.0
# 565 The Silent Sister
        (Riley MacPherson, #1) Rating: 0.0
# 566 The Judge's List
        (The Whistler, #2) Rating: 0.0
# 567 Danse Macabre Rating: 0.0
# 568 Just for the Summer
        (Part of Your World, #3) Rating: 0.0
# 569 The Client Rating: 0.0
# 570 The Book of Lost Names Rating: 0.0
# 571 Watership Down
        (Watership Down, #1) Rating: 0.0
# 572 The Adventures of Tom Sawyer Rating: 0.0
# 573 Perelandra
        (The Space Trilogy, #2) Rating: 0.0
# 574 Muse of Nightmares
        (Strange the Dreamer, #2) Rating: 0.0
# 575 The Last Battle
        (Chronicles of Narnia, #7) Rating: 0.0
# 576 Flipped Rating: 0.0
# 577 When the Wind Blows
        (When the Wind Blows, #1) Rating: 0.0
# 578 The Witch's Daughter
        (The Witch's Daughter, #1) Rating: 0.0
# 579 Pride and Prejudice and Zombies
       

...

# 3153 Monday Mourning
        (Temperance Brennan, #7) Rating: 0.0
# 3154 Unnatural Exposure
        (Kay Scarpetta, #8) Rating: 0.0
# 3155 Oh, the Places Youâ€™ll Go! Rating: 0.0
# 3156 The Ultimate Hitchhiker’s Guide to the Galaxy
        (Hitchhiker's Guide to the Galaxy, #1-5) Rating: 0.0
# 3157 The Next Mrs. Parrish
        (Mrs. Parrish, #2) Rating: 0.0
# 3158 The Talisman
        (The Talisman, #1) Rating: 0.0
# 3159 The Overlook
        (Harry Bosch, #13; Harry Bosch Universe, #18) Rating: 0.0
# 3160 The Vampire Lestat
        (The Vampire Chronicles, #2) Rating: 0.0
# 3161 Brokeback Mountain Rating: 0.0
# 3162 Forever in Blue: The Fourth Summer of the Sisterhood
        (Sisterhood, #4) Rating: 0.0
# 3163 The Castle of Llyr
        (The Chronicles of Prydain, #3) Rating: 0.0
# 3164 On a Quiet Street Rating: 0.0
# 3165 Hard-Boiled Wonderland and the End of the World Rating: 0.0
# 3166 The Circle
        (The Circle, #1) Rating: 0.0
# 3167 Good in Bed
        (Cannie Shapiro, #

        (Michael Bennett, #2) Rating: 0.0
# 3328 All the Ugly and Wonderful Things Rating: 0.0
# 3329 Winter's Heart
        (The Wheel of Time, #9) Rating: 0.0
# 3330 Barrel Fever: Stories and Essays Rating: 0.0
# 3331 Towers of Midnight
        (The Wheel of Time, #13) Rating: 0.0
# 3332 Burn Rating: 0.0
# 3333 Ham on Rye Rating: 0.0
# 3334 Severance Rating: 0.0
# 3335 The Woman Upstairs Rating: 0.0
# 3336 Outliers: The Story of Success Rating: 0.0
# 3337 The Light Fantastic
        (Discworld, #2; Rincewind, #2) Rating: 0.0
# 3338 Razorblade Tears Rating: 0.0
# 3339 Something Blue
        (Darcy & Rachel, #2) Rating: 0.0
# 3340 The Family Across the Street Rating: 0.0
# 3341 Disappearing Earth Rating: 0.0
# 3342 The Story of the Lost Child
        (Neapolitan Novels, #4) Rating: 0.0
# 3343 The Power That Preserves
        (The Chronicles of Thomas Covenant the Unbeliever, #3) Rating: 0.0
# 3344 The Ten Thousand Doors of January Rating: 0.0
# 3345 Forever
        (The Wolves of Mercy

        (Discworld, #9; Rincewind, #4) Rating: 0.0
# 3499 The Many Lives of Mama Love: A Memoir of Lying, Stealing, Writing, and Healing Rating: 0.0
# 3500 The Killing Dance
        (Anita Blake, Vampire Hunter, #6) Rating: 0.0
# 3501 Bringing Up BÃ©bÃ©: One American Mother Discovers the Wisdom of French Parenting Rating: 0.0
# 3502 Uncle Vanya Rating: 0.0
# 3503 Nora Webster Rating: 0.0
# 3504 A Good Girl's Guide to Murder
        (A Good Girl's Guide to Murder, #1) Rating: 0.0
# 3505 The Mysterious Affair at Styles
        (Hercule Poirot, #1) Rating: 0.0
# 3506 A Lesson Before Dying Rating: 0.0
# 3507 Resting Scrooge Face Rating: 0.0
# 3508 A Darkness More Than Night
        (Harry Bosch, #7; Harry Bosch Universe, #10) Rating: 0.0
# 3509 The Cardinal of the Kremlin
        (Jack Ryan, #4) Rating: 0.0
# 3510 Peter and the Starcatchers
        (Peter and the Starcatchers, #1) Rating: 0.0
# 3511 Pandora
        (New Tales of the Vampires, #1) Rating: 0.0
# 3512 Born to Run: A Hidden Tr

# 3683 The Mothers Rating: 0.0
# 3684 The Girl Who Takes an Eye for an Eye
        (Millennium, #5) Rating: 0.0
# 3685 An Anonymous Girl Rating: 0.0
# 3686 The Tales of Beedle the Bard Rating: 0.0
# 3687 The Hero of Ages
        (Mistborn, #3) Rating: 0.0
# 3688 A Winter in New York Rating: 0.0
# 3689 The Reptile Room
        (A Series of Unfortunate Events, #2) Rating: 0.0
# 3690 Cat on a Hot Tin Roof Rating: 0.0
# 3691 Dune Messiah
        (Dune, #2) Rating: 0.0
# 3692 Lirael
        (Abhorsen, #2) Rating: 0.0
# 3693 Slow Dance Rating: 0.0
# 3694 Congo Rating: 0.0
# 3695 Xenocide
        (Ender's Saga, #3) Rating: 0.0
# 3696 Brooklyn
        (Eilis Lacey, #1) Rating: 0.0
# 3697 Fast Food Nation: The Dark Side of the All-American Meal Rating: 0.0
# 3698 An Unquiet Mind: A Memoir of Moods and Madness Rating: 0.0
# 3699 The Bean Trees
        (Greer Family, #1) Rating: 0.0
# 3700 First Comes Love Rating: 0.0
# 3701 The Extraordinary Life of Sam Hell Rating: 0.0
# 3702 The Winds of War
 

# 3890 Special Topics in Calamity Physics Rating: 0.0
# 3891 Wandering Stars Rating: 0.0
# 3892 Wicked Lovely
        (Wicked Lovely, #1) Rating: 0.0
# 3893 David and Goliath: Underdogs, Misfits, and the Art of Battling Giants Rating: 0.0
# 3894 The World of Winnie-the-Pooh
        (Winnie-the-Pooh, #1-2) Rating: 0.0
# 3895 Never Have I Ever Rating: 0.0
# 3896 Pretty Little Wife Rating: 0.0
# 3897 American Gods Rating: 0.0
# 3898 Did You Ever Have a Family Rating: 0.0
# 3899 Heidi Rating: 0.0
# 3900 Sizzling Sixteen
        (Stephanie Plum, #16) Rating: 0.0
# 3901 The Story of Edgar Sawtelle Rating: 0.0
# 3902 Slouching Towards Bethlehem Rating: 0.0
# 3903 Nine Lives Rating: 0.0
# 3904 Pictures of You Rating: 0.0
# 3905 A River in Darkness: One Man's Escape from North Korea Rating: 0.0
# 3906 The Secret Place
        (Dublin Murder Squad, #5) Rating: 0.0
# 3907 Fantasy Lover
        (Hunter Legends, #1) Rating: 0.0
# 3908 The Woman in Me Rating: 0.0
# 3909 In Five Years Rating: 0.0
# 3

In [17]:
#doing masked autoencoder
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset, random_split

# Mask for observed values (1 for observed, 0 for missing)
ratings_torch = torch.tensor(ratings).float()
mask = (ratings_torch != 0).float()
mask_tensor = torch.tensor(mask)


#Define autoencoder
class SparseAutoencoder(nn.Module):
    def __init__(self, num_items, latent_dim):
        super(SparseAutoencoder, self).__init__()
        self.encoder = nn.Linear(num_items, latent_dim)
        self.decoder = nn.Linear(latent_dim, num_items)
        
    def forward(self, x):
        encoded = torch.relu(self.encoder(x))
        decoded = self.decoder(encoded)
        # Scale sigmoid output to [1, 5]
        return 1 + 4 * torch.sigmoid(decoded)
        return decoded

    
#initialize the model
num_users, num_items = ratings_torch.shape
np.save("num_users.npy", np.array(num_users))
np.save("num_items.npy", np.array(num_items))
latent_dim = 100 # Number of latent features

model = SparseAutoencoder(num_items, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Use MSE loss but only consider observed values
def masked_mse_loss(reconstructed, original, mask):
    loss = ((reconstructed - original) ** 2) * mask
    return loss.sum() / mask.sum()

#break up data into train and val
dataset = TensorDataset(ratings_torch, mask_tensor) #keeping the mask
print(len(dataset))
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print("len(train_loader) = ", len(train_loader))
print("len(val_loader) = ", len(val_loader))

#train the model
epochs = 5000
best_loss = 10e10
counter = 0
for epoch in range(epochs):
    train_loss = 0.0
    for inputs, mask in train_loader:
        model.train()
        optimizer.zero_grad()

        # Forward pass
        reconstructed = model(inputs)
        loss = masked_mse_loss(reconstructed, inputs, mask)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    train_loss /= len(train_loader)
    
    if (epoch + 1) % 10 == 0:
#         print(f"Epoch {epoch + 1}/{epochs}, Loss: {train_loss}")
        #check validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, mask in val_loader:
                outputs = model(inputs)
                loss = masked_mse_loss(outputs, inputs, mask)
                val_loss += loss.item()
                

        val_loss /= len(val_loader)
        

        print(f"Epoch {epoch + 1} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")
        if val_loss < best_loss: #if improve then save
            #save model 
            torch.save(model.state_dict(), "model.pkl")
            best_loss = val_loss
            print("Model saved to model.pkl.")
            counter = 0
        else:
            counter += 10
            
    if counter > 200:
        print("Done training because of no improvement.")
        break
        
            


/tmp/ipykernel_271/4276932551.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask_tensor = torch.tensor(mask)


1002
len(train_loader) =  26
len(val_loader) =  7
Epoch 10 - Train Loss: 0.6037 - Val Loss: 1.2650
Model saved to model.pkl.
Epoch 20 - Train Loss: 0.5123 - Val Loss: 1.1434
Model saved to model.pkl.
Epoch 30 - Train Loss: 0.4858 - Val Loss: 1.0959
Model saved to model.pkl.
Epoch 40 - Train Loss: 0.5481 - Val Loss: 1.1190
Epoch 50 - Train Loss: 0.5366 - Val Loss: 1.1204
Epoch 60 - Train Loss: 0.5390 - Val Loss: 1.1029
Epoch 70 - Train Loss: 0.5111 - Val Loss: 1.1021
Epoch 80 - Train Loss: 0.5598 - Val Loss: 1.1166
Epoch 90 - Train Loss: 0.5613 - Val Loss: 1.1115
Epoch 100 - Train Loss: 0.6008 - Val Loss: 1.1137
Epoch 110 - Train Loss: 0.6012 - Val Loss: 1.1043
Epoch 120 - Train Loss: 0.5705 - Val Loss: 1.0992
Epoch 130 - Train Loss: 0.5899 - Val Loss: 1.1086
Epoch 140 - Train Loss: 0.6118 - Val Loss: 1.1061
Epoch 150 - Train Loss: 0.6263 - Val Loss: 1.1019
Epoch 160 - Train Loss: 0.6008 - Val Loss: 1.0980
Epoch 170 - Train Loss: 0.6686 - Val Loss: 1.0957
Model saved to model.pkl.
Epoch

In [18]:
num_users, num_items

(1002, 4107)

In [19]:
print(sum(sum(mask)))
print(mask.shape)
print(mask.shape[0] * mask.shape[1])
print(sum(sum(mask)) / (mask.shape[0] * mask.shape[1]))



tensor(470.)
torch.Size([9, 4107])
36963
tensor(0.0127)


In [20]:
#Evaulating the model
model.eval()
with torch.no_grad():
    reconstructed = model(ratings_torch)

# Fill missing values in the original matrix
filled_data = ratings_torch.clone()
filled_data[mask_tensor == 0] = reconstructed[mask_tensor == 0]

print("Original Data:\n", ratings_torch)
print("Reconstructed Data:\n", reconstructed)
print("Filled Data:\n", filled_data)


Original Data:
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Reconstructed Data:
 tensor([[4.1460, 3.5425, 4.5695,  ..., 4.4060, 3.3533, 3.8514],
        [4.1460, 3.5425, 4.5695,  ..., 4.4060, 3.3533, 3.8514],
        [4.1460, 3.5425, 4.5695,  ..., 4.4060, 3.3533, 3.8514],
        ...,
        [5.0000, 5.0000, 5.0000,  ..., 5.0000, 1.0000, 5.0000],
        [4.7738, 2.9818, 4.9949,  ..., 4.9431, 1.5735, 4.8015],
        [4.1460, 3.5425, 4.5695,  ..., 4.4060, 3.3533, 3.8514]])
Filled Data:
 tensor([[4.1460, 3.5425, 4.5695,  ..., 4.4060, 3.3533, 3.8514],
        [4.1460, 3.5425, 4.5695,  ..., 4.4060, 3.3533, 3.8514],
        [4.1460, 3.5425, 4.5695,  ..., 4.4060, 3.3533, 3.8514],
        ...,
        [5.0000, 5.0000, 5.0000,  ..., 5.0000, 1.0000, 5.0000],
        [4.7738, 2.9818, 4.9

In [21]:
print(reconstructed[0].numpy())
print(ratings_torch[0].numpy())

print((reconstructed[0].numpy()-ratings_torch[0].numpy())/ratings_torch[0].numpy())

[4.1459517 3.542534  4.5694876 ... 4.4059744 3.3532996 3.851355 ]
[0. 0. 0. ... 0. 0. 0.]
[inf inf inf ... inf inf inf]


/tmp/ipykernel_271/2166569457.py:4: RuntimeWarning: divide by zero encountered in divide
  print((reconstructed[0].numpy()-ratings_torch[0].numpy())/ratings_torch[0].numpy())


In [22]:
pred_ratings_list = reconstructed[0].detach().numpy()

#give a list sorted out with books you've already read:
sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
list_num = 1
for idx in sorted_indices: 
#     print("ratings_matrix[user_id, idx]= ", ratings_matrix[user_id, idx])
    if  (ratings_matrix[user_id, idx] > 0) or(np.isnan(pred_ratings_list[idx])) :
        continue
    print("#", list_num , titles[idx], " - Predicted Rating:", round(pred_ratings_list[idx], 1))
    list_num += 1

Top books are:
# 1 A Wind in the Door
        (Time Quintet, #2)  - Predicted Rating: 5.0
# 2 Long Walk to Freedom  - Predicted Rating: 5.0
# 3 The Door  - Predicted Rating: 5.0
# 4 The Happiest Man on Earth  - Predicted Rating: 5.0
# 5 The Reformatory  - Predicted Rating: 5.0
# 6 Memorial Day
        (Mitch Rapp, #7)  - Predicted Rating: 5.0
# 7 All My Knotted-Up Life: A Memoir  - Predicted Rating: 5.0
# 8 So You Want to Talk About Race  - Predicted Rating: 5.0
# 9 The Hunger Games Trilogy Boxset
        (The Hunger Games, #1-3)  - Predicted Rating: 5.0
# 10 Gemina
        (The Illuminae Files, #2)  - Predicted Rating: 5.0
# 11 The Borrowed Life of Frederick Fife  - Predicted Rating: 5.0
# 12 Once We Were Home  - Predicted Rating: 5.0
# 13 Queen of Sorcery
        (The Belgariad #2)  - Predicted Rating: 5.0
# 14 Dragons of Spring Dawning
        (Dragonlance: Chronicles, #3)  - Predicted Rating: 5.0
# 15 Hunger: A Memoir of (My) Body  - Predicted Rating: 5.0
# 16 The German Wife  - Pr

        (Letters of Enchantment, #2)  - Predicted Rating: 4.7
# 159 Faith of the Fallen
        (Sword of Truth, #6)  - Predicted Rating: 4.7
# 160 Ramona Quimby, Age 8
        (Ramona, #6)  - Predicted Rating: 4.7
# 161 The Girl with the Louding Voice  - Predicted Rating: 4.7
# 162 The Sandman, Vol. 4: Season of Mists  - Predicted Rating: 4.7
# 163 Dad Is Fat  - Predicted Rating: 4.7
# 164 The Master and Margarita  - Predicted Rating: 4.7
# 165 Queen of Shadows
        (Throne of Glass, #4)  - Predicted Rating: 4.7
# 166 A Twisted Love Story  - Predicted Rating: 4.7
# 167 The Incredible Journey  - Predicted Rating: 4.6
# 168 Clockwork Prince
        (The Infernal Devices, #2)  - Predicted Rating: 4.6
# 169 The Complete Works  - Predicted Rating: 4.6
# 170 Always and Forever, Lara Jean
        (To All the Boys I've Loved Before, #3)  - Predicted Rating: 4.6
# 171 Rita Hayworth and Shawshank Redemption  - Predicted Rating: 4.6
# 172 Absolute Power  - Predicted Rating: 4.6
# 173 The Son 

        (Left Behind, #4)  - Predicted Rating: 4.5
# 287 In Defense of Food: An Eater's Manifesto  - Predicted Rating: 4.5
# 288 The Love Dare  - Predicted Rating: 4.5
# 289 The Love Elixir of Augusta Stern  - Predicted Rating: 4.5
# 290 The Fire Next Time  - Predicted Rating: 4.5
# 291 The Mysterious Benedict Society
        (The Mysterious Benedict Society, #1)  - Predicted Rating: 4.5
# 292 Foster  - Predicted Rating: 4.5
# 293 The Borrowers
        (The Borrowers, #1)  - Predicted Rating: 4.5
# 294 The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics  - Predicted Rating: 4.5
# 295 The Lottery  - Predicted Rating: 4.5
# 296 First Lord's Fury
        (Codex Alera, #6)  - Predicted Rating: 4.5
# 297 The Complete Tales  - Predicted Rating: 4.5
# 298 Total Control  - Predicted Rating: 4.5
# 299 All the Sinners Bleed  - Predicted Rating: 4.5
# 300 Canada  - Predicted Rating: 4.5
# 301 The House of Hades
        (The Heroes of Olympus, #4)  - Pred

# 469 Notorious RBG: The Life and Times of Ruth Bader Ginsburg  - Predicted Rating: 4.4
# 470 Anne of Windy Poplars
        (Anne of Green Gables, #4)  - Predicted Rating: 4.4
# 471 Faefever
        (Fever, #3)  - Predicted Rating: 4.4
# 472 Halfway to the Grave
        (Night Huntress, #1)  - Predicted Rating: 4.4
# 473 Natchez Burning
        (Penn Cage, #4)  - Predicted Rating: 4.4
# 474 Carrie Soto Is Back  - Predicted Rating: 4.4
# 475 The Help  - Predicted Rating: 4.4
# 476 Midnight in Chernobyl: The Untold Story of the World's Greatest Nuclear Disaster  - Predicted Rating: 4.4
# 477 Saga, Volume 1  - Predicted Rating: 4.4
# 478 Brideshead Revisited  - Predicted Rating: 4.4
# 479 The One Hundred Years of Lenni and Margot  - Predicted Rating: 4.4
# 480 The Gift of the Magi  - Predicted Rating: 4.4
# 481 Light in August  - Predicted Rating: 4.4
# 482 The Book of Longings  - Predicted Rating: 4.4
# 483 The Perfect Couple
        (Nantucket, #3)  - Predicted Rating: 4.4
# 484 Saint X

# 601 We Solve Murders
        (We Solve Murders, #1)  - Predicted Rating: 4.3
# 602 The Wedding People  - Predicted Rating: 4.3
# 603 The House at Pooh Corner
        (Winnie-the-Pooh, #2)  - Predicted Rating: 4.3
# 604 The Queen of Nothing
        (The Folk of the Air, #3)  - Predicted Rating: 4.3
# 605 Blindsighted
        (Grant County, #1)  - Predicted Rating: 4.3
# 606 Tick Tock
        (Michael Bennett, #4)  - Predicted Rating: 4.3
# 607 Wild Swans: Three Daughters of China  - Predicted Rating: 4.3
# 608 Walk Two Moons  - Predicted Rating: 4.3
# 609 The Book of Laughter and Forgetting  - Predicted Rating: 4.3
# 610 The Code Breaker: Jennifer Doudna, Gene Editing, and the Future of the Human Race  - Predicted Rating: 4.3
# 611 The World Is Flat: A Brief History of the Twenty-first Century  - Predicted Rating: 4.3
# 612 The Joy of Cooking  - Predicted Rating: 4.3
# 613 Blood Rites
        (The Dresden Files, #6)  - Predicted Rating: 4.3
# 614 The Four Winds  - Predicted Rating: 4.

...

# 3316 Lullaby  - Predicted Rating: 3.4
# 3317 Circling the Sun  - Predicted Rating: 3.4
# 3318 Pop Goes the Weasel
        (Alex Cross, #5)  - Predicted Rating: 3.4
# 3319 A Constellation of Vital Phenomena  - Predicted Rating: 3.4
# 3320 Graceling
        (Graceling Realm, #1)  - Predicted Rating: 3.4
# 3321 The Island of Dr. Moreau  - Predicted Rating: 3.4
# 3322 Paradise Lost  - Predicted Rating: 3.4
# 3323 The Midnight Line
        (Jack Reacher, #22)  - Predicted Rating: 3.4
# 3324 Survive the Night  - Predicted Rating: 3.4
# 3325 Blackout
        (All Clear, #1)  - Predicted Rating: 3.4
# 3326 Finding Cinderella
        (Hopeless, #2.5)  - Predicted Rating: 3.4
# 3327 A Fatal Grace
        (Chief Inspector Armand Gamache, #2)  - Predicted Rating: 3.4
# 3328 The Tiger's Wife  - Predicted Rating: 3.4
# 3329 The 5th Wave
        (The 5th Wave, #1)  - Predicted Rating: 3.4
# 3330 Batman: Year One  - Predicted Rating: 3.4
# 3331 Siddhartha  - Predicted Rating: 3.4
# 3332 Simple Geniu

# 3445 One Dark Window
        (The Shepherd King, #1)  - Predicted Rating: 3.3
# 3446 A Town Like Alice  - Predicted Rating: 3.3
# 3447 Rosie and the Dreamboat
        (The Improbable Meet-Cute, #3)  - Predicted Rating: 3.3
# 3448 Inca Gold
        (Dirk Pitt, #12)  - Predicted Rating: 3.3
# 3449 The Boy in the Striped Pajamas  - Predicted Rating: 3.3
# 3450 The Lion's Den  - Predicted Rating: 3.3
# 3451 Blue Bloods
        (Blue Bloods, #1)  - Predicted Rating: 3.3
# 3452 The Gunslinger
        (The Dark Tower, #1)  - Predicted Rating: 3.3
# 3453 The Midnight Feast  - Predicted Rating: 3.3
# 3454 Vanity Fair  - Predicted Rating: 3.3
# 3455 Guards! Guards!
        (Discworld, #8; City Watch, #1)  - Predicted Rating: 3.3
# 3456 All the Missing Girls  - Predicted Rating: 3.3
# 3457 Make Way for Ducklings  - Predicted Rating: 3.3
# 3458 The Dilemma  - Predicted Rating: 3.3
# 3459 The Mote in God's Eye
        (Moties, #1)  - Predicted Rating: 3.3
# 3460 Disappearing Earth  - Predicted Ra

# 3631 One of Us Is Dead  - Predicted Rating: 3.2
# 3632 Go, Dog. Go!  - Predicted Rating: 3.2
# 3633 Carpe Jugulum
        (Discworld, #23; Witches, #6)  - Predicted Rating: 3.2
# 3634 Prey  - Predicted Rating: 3.2
# 3635 The Boxcar Children
        (The Boxcar Children, #1)  - Predicted Rating: 3.2
# 3636 The Physick Book of Deliverance Dane
        (The Physick Book, #1)  - Predicted Rating: 3.2
# 3637 The Secret Adversary
        (Tommy and Tuppence Mysteries, #1)  - Predicted Rating: 3.2
# 3638 The Vacationers  - Predicted Rating: 3.2
# 3639 Hook, Line, and Sinker
        (Bellinger Sisters, #2)  - Predicted Rating: 3.2
# 3640 Maskerade
        (Discworld, #18; Witches, #5)  - Predicted Rating: 3.2
# 3641 The Maze of Bones
        (The 39 Clues, #1)  - Predicted Rating: 3.2
# 3642 The House Girl  - Predicted Rating: 3.2
# 3643 A Portrait of the Artist as a Young Man  - Predicted Rating: 3.2
# 3644 The Woman in Cabin 10  - Predicted Rating: 3.2
# 3645 Heart of Darkness  - Predicted

# 3764 To Rise Again at a Decent Hour  - Predicted Rating: 3.0
# 3765 Tweak: Growing Up On Methamphetamines  - Predicted Rating: 3.0
# 3766 The Rum Diary  - Predicted Rating: 3.0
# 3767 Gild
        (The Plated Prisoner, #1)  - Predicted Rating: 3.0
# 3768 No, David!  - Predicted Rating: 3.0
# 3769 Resting Scrooge Face  - Predicted Rating: 3.0
# 3770 Winter's Heart
        (The Wheel of Time, #9)  - Predicted Rating: 3.0
# 3771 When She Returned  - Predicted Rating: 3.0
# 3772 All Clear
        (All Clear, #2)  - Predicted Rating: 3.0
# 3773 On Basilisk Station
        (Honor Harrington, #1)  - Predicted Rating: 3.0
# 3774 The Halloween Tree  - Predicted Rating: 3.0
# 3775 The Husband  - Predicted Rating: 3.0
# 3776 Hallowe'en Party
        (Hercule Poirot, #41)  - Predicted Rating: 3.0
# 3777 The Killing Dance
        (Anita Blake, Vampire Hunter, #6)  - Predicted Rating: 3.0
# 3778 Seating Arrangements  - Predicted Rating: 3.0
# 3779 Escape  - Predicted Rating: 3.0
# 3780 Alanna: The

# 3947 Special Topics in Calamity Physics  - Predicted Rating: 2.7
# 3948 Go Ask Alice  - Predicted Rating: 2.7
# 3949 The Senator's Wife  - Predicted Rating: 2.7
# 3950 Savage Run
        (Joe Pickett, #2)  - Predicted Rating: 2.7
# 3951 Incubus Dreams
        (Anita Blake, Vampire Hunter, #12)  - Predicted Rating: 2.7
# 3952 The Replacement  - Predicted Rating: 2.7
# 3953 The Butterfly Garden
        (The Collector, #1)  - Predicted Rating: 2.7
# 3954 Reflected in You
        (Crossfire, #2)  - Predicted Rating: 2.7
# 3955 Cradle and All  - Predicted Rating: 2.7
# 3956 When No One Is Watching  - Predicted Rating: 2.7
# 3957 Are You There God? Itâ€™s Me, Margaret  - Predicted Rating: 2.7
# 3958 Entwined with You
        (Crossfire, #3)  - Predicted Rating: 2.7
# 3959 Visions of Sugar Plums
        (Stephanie Plum, #8.5)  - Predicted Rating: 2.7
# 3960 Micah
        (Anita Blake, Vampire Hunter, #13)  - Predicted Rating: 2.7
# 3961 Confess  - Predicted Rating: 2.7
# 3962 The Hangman's 

In [23]:
dfghj
import torch
from sklearn.model_selection import KFold

# Mask for observed values (1 for observed, 0 for missing)
ratings_torch = torch.tensor(ratings).float()
mask = (ratings_torch != 0).float()
print(mask)


#Define autoencoder
class SparseAutoencoder(nn.Module):
    def __init__(self, num_items, latent_dim):
        super(SparseAutoencoder, self).__init__()
        self.encoder = nn.Linear(num_items, latent_dim)
        self.decoder = nn.Linear(latent_dim, num_items)
        
    def forward(self, x):
        encoded = torch.relu(self.encoder(x))
        decoded = self.decoder(encoded)
        # Scale sigmoid output to [1, 5]
        return 1 + 4 * torch.sigmoid(decoded)
        return decoded

    
#initialize the model
num_users, num_items = ratings_torch.shape
np.save("num_users.npy", np.array(num_users))
np.save("num_items.npy", np.array(num_items))

for latent_dim in [2, 5, 10, 20, 40, 50, 75, 100]:
    print("latent_dim = ", latent_dim)
# latent_dim = 20  # Number of latent features

    model = SparseAutoencoder(num_items, latent_dim)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Define your model, loss function, and optimizer
    # Assuming model, ratings_torch, mask, and optimizer are already defined

    epochs = 1000
    k_folds = 5  # Number of folds for cross-validation
    kf = KFold(n_splits=k_folds, shuffle=True)

    # Store the losses for each fold
    fold_losses = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(ratings_torch)):
#         print(f"\nFold {fold + 1}/{k_folds}")

        # Split the data into training and validation sets
        train_ratings = ratings_torch[train_idx]
        val_ratings = ratings_torch[val_idx]
        train_mask = mask[train_idx]
        val_mask = mask[val_idx]

        # Initialize a new model for each fold
        model = SparseAutoencoder(num_items, latent_dim)
    #     optimizer = optim.Adam(model.parameters(), lr=0.01)

        # Re-initialize optimizer for each fold
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        # Train the model
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()

            # Forward pass for training
            reconstructed = model(train_ratings)
            loss = masked_mse_loss(reconstructed, train_ratings, train_mask)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

#             if (epoch + 1) % 1000 == 0:
#                 print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")

        # Evaluate the model on the validation set
        model.eval()
        with torch.no_grad():
            reconstructed_val = model(val_ratings)
            val_loss = masked_mse_loss(reconstructed_val, val_ratings, val_mask)

        print(f"Validation Loss for Fold {fold + 1}: {val_loss.item():.4f}")

        # Store the validation loss for this fold
        fold_losses.append(val_loss.item())

    # Print the average validation loss after all folds
    print(f"\nAverage Validation Loss across all folds: {sum(fold_losses)/k_folds:.4f}")


NameError: name 'dfghj' is not defined

In [ ]:
fgh

In [ ]:
ratings_matrix[user_id]

In [ ]:
# Convert the filled ratings data into a numpy array
ratings_matrix = ratings_df.values

# Initialize KNN (using user-based KNN)
import math
knn = NearestNeighbors(n_neighbors=math.ceil(num_users/10), metric='cosine')  # Using cosine similarity
knn.fit(ratings_matrix)

# Example: Predict rating for User 0 on Item 2
user_id = 0  # Index of user in the matrix
item_id = 9  # Index of item to predict rating for

# Get the nearest neighbors for user 0 (excluding the user itself)
distances, indices = knn.kneighbors([ratings_matrix[user_id]])

pred_ratings_list = np.array([])
rankings_list = np.array([])
for item_id in range(num_titles):
    # Get the ratings for the neighbors on item 2
    neighbor_ratings = np.array([ratings_matrix[i, item_id] for i in indices[0] if not np.isnan(ratings_matrix[i, item_id])])
    print(ratings_matrix[:, item_id])
    print(neighbor_ratings)
    ghjk
    
    predicted_rating = np.mean(neighbor_ratings[np.nonzero(neighbor_ratings)])
    rankings = np.sum(neighbor_ratings[np.nonzero(neighbor_ratings)])

#     print(predicted_rating)

    pred_ratings_list = np.append(pred_ratings_list, predicted_rating)
    rankings_list = np.append(rankings_list, rankings)
#     print(f"Predicted rating for User {user_id} on Item {item_id}: {predicted_rating}")
    
best_book_rating = np.max(pred_ratings_list)
best_book_idx = np.argmax(pred_ratings_list)

sorted_indices = np.argsort(pred_ratings_list)[::-1]
print("Top books are:")
for i, idx in enumerate(sorted_indices): 
    print("#", (i+1) , titles[idx], "Rating:", round(pred_ratings_list[idx], 1))